In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


np.random.seed(42)

train = pd.read_csv('train.csv.zip', compression='zip')
x_train = train.drop(['id', 'species'], axis=1).values
le = LabelEncoder().fit(train['species'])
y_train = le.transform(train['species'])

scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)

params = {'C':[100, 1000], 'tol': [0.001, 0.0001]}
log_reg = LogisticRegression(solver='lbfgs', multi_class='multinomial')
clf = GridSearchCV(log_reg, params, scoring='log_loss', refit='True', n_jobs=-1, cv=5)
clf.fit(x_train, y_train)

print("best params: " + str(clf.best_params_))
for params, mean_score, scores in clf.grid_scores_:
  print("%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std(), params))
  print(scores)

test = pd.read_csv('test.csv.zip', compression='zip')
test_ids = test.pop('id')
x_test = test.values
x_test = scaler.transform(x_test)

y_test = clf.predict_proba(x_test)

submission = pd.DataFrame(y_test, index=test_ids, columns=le.classes_)
submission.to_csv('submission_log_reg.csv')

/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py:136: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/opt/conda/lib/python3.5/site-packages/sklearn/metrics/scorer.py

best params: {'tol': 0.001, 'C': 1000}
-0.044 (+/-0.020) for {'tol': 0.001, 'C': 100}
[-0.04893957 -0.02490256 -0.07837384 -0.04507673 -0.02316519]
-0.044 (+/-0.020) for {'tol': 0.0001, 'C': 100}
[-0.04937695 -0.02511783 -0.07826285 -0.04507673 -0.02298998]
-0.041 (+/-0.023) for {'tol': 0.001, 'C': 1000}
[-0.04247012 -0.01781544 -0.08315391 -0.039078   -0.02200079]
-0.041 (+/-0.024) for {'tol': 0.0001, 'C': 1000}
[-0.04459837 -0.01847142 -0.08396316 -0.04218453 -0.0173108 ]
